<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خونه جدید‌؟
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این تمرین می‌خواهیم به‌صورت عملی با نحوه‌ی پیاده‌سازی یک شبکه‌ی عصبی عمیق در <code>keras</code> آشنا شویم. کار خود را با خوانش داده‌ها و تحلیل اکتشافی آن شروع کرده، سپس با استفاده از پیش‌پردازش‌های مختلف، داده‌ها را به‌شکلی که مناسب استفاده در یک مدل یادگیری عمیق باشد آماده می‌کنیم و در نهایت به ساخت و آموزش یک شبکه‌ی عصبی می‌پردازیم. مجموعه‌داده‌ای که از آن استفاده خواهیم کرد شامل اطلاعاتی مربوط به حیوانات خانگی است. هدف این مسئله این است که پیش‌بینی کنیم آیا یک حیوان خانگی برای خود خانه‌ی جدیدی پیدا می‌کند یا خیر.
</font>
</p>

In [58]:
import numpy as np
import pandas as pd

In [59]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [60]:
import keras
print(keras.__version__)

3.1.1


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
مشخصات ویژگی‌های مجموعه‌داده‌ای که در دسترس شما قرار گرفته در جداول زیر آمده است.
ابتدا مجموعه‌داده‌ی آموزشی و آزمون را از دو فایل <code>csv</code> موجود در پوشه‌ی تمرین بخوانید.
<br>
    توجه داشته‌باشد در مجموعه‌داده‌ی آزمون، ستون <code>AdoptionSpeed</code> وجود ندارد، زیرا در ادامه از این ستون برای ساخت متغیر هدف مسئله استفاده خواهیم کرد.
</font>
</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|ستون|توضیحات|
|:------:|:---:|
|<code>Type</code>| نوع حیوان|
|<code>Age</code>| سن حیوان خانگی (به ماه) هنگام لیست شدن|
|<code>Breed1</code>|نژاد اصلی حیوان خانگی|
|<code>Gender</code>|جنسیت حیوان خانگی|
|<code>Color1</code>|رنگ اصلی حیوان خانگی|
|<code>Color2</code>|رنگ دوم حیوان خانگی (در صورت وجود)|
|<code>MaturitySize</code>|اندازه|
|<code>FurLength</code>|طول خز|
|<code>Vaccinated</code>|وضعیت واکسینه شدن|
|<code>Sterilized</code>|وضعیت انگل‌زدایی شدن|
|<code>Health</code>|وضعیت سلامتی|
|<code>Fee</code>|هزینه|
|<code>Description</code>|توضیحات|
|<code>PhotoAmt</code>|تعداد عکس بارگذاری شده|
|<code>AdoptionSpeed</code>|سرعت پذیرش (توضیح این اعداد در ادامه آمده است)|

<br>

|توضیحات|سرعت پذیرش|
|:------:|:---:|
|حیوان خانگی در همان روزی که لیست شده به سرپرستی پذیرفته شد.|<code>0</code>|
|	حیوان خانگی بین ۱ تا ۷ روز (هفته اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>1</code>|
|حیوان خانگی بین ۸ تا ۳۰ روز (ماه اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>2</code>|
|حیوان خانگی بین ۳۱ تا ۹۰ روز (ماه دوم و سوم) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>3</code>|
|پس از ۱۰۰ روز لیست شدن، حیوان خانگی به سرپرستی پذیرفته نشد.|<code>4</code>|

</font>
</div>
</center>

In [61]:
# Load the train and test data
train = pd.read_csv('data/petfinder_train.csv')
test = pd.read_csv('data/petfinder_test.csv')

In [62]:
train.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           10537 non-null  object
 1   Age            10537 non-null  int64 
 2   Breed1         10537 non-null  object
 3   Gender         10537 non-null  object
 4   Color1         10537 non-null  object
 5   Color2         10537 non-null  object
 6   MaturitySize   10537 non-null  object
 7   FurLength      10537 non-null  object
 8   Vaccinated     10537 non-null  object
 9   Sterilized     10537 non-null  object
 10  Health         10537 non-null  object
 11  Fee            10537 non-null  int64 
 12  Description    10528 non-null  object
 13  PhotoAmt       10537 non-null  int64 
 14  AdoptionSpeed  10537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.2+ MB


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش داده‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که می‌دانید قبل از ساخت و آموزش مدل، برای آن‌که داده‌ها مناسب ورودی دادن به مدل باشند معمولاً نیاز است تغیراتی را بر روی مجموعه‌داده اعمال کنیم. گام نخست در پیش‌پردازش داده‌ها، بررسی مقادیر گم‌شده در مجموعه‌داده است که طبق خروجی سلول قبل مشاهده کردیم در این مجموعه‌داده، داده‌ی گم‌شده نداریم، بنابراین به سراغ گام‌های بعدی می‌رویم.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
<span style="color:red"><b>توجه:</b></span> جهت بررسی پیش‌پردازش انجام‌شده‌ بر روی مجموعه‌داده، دیتافرم <code>test</code> نیز مورد بررسی قرار خواهد گرفت، بنابراین لطفاً پیش‌پردازش یا تغییری بیشتر از آن‌چه از شما خواسته شده انجام ندهید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که گفتیم هدف این مسئله، پیش‌بینی پذیرش یا عدم پذیرش حیوان خانگی است، بنابراین باید در مجموعه‌داده‌‌ها یک ستون جدید با اسم <code>Target</code> اضافه کنیم تا از آن به‌عنوان متغیر هدف مسئله استفاده کنیم. مقادیر این ستون تنها در صورتی <code>True</code> است که حیوان خانگی پذیرش شده باشد. در این مرحله با استفاده ستون <code>AdoptionSpeed</code> ستون <code>Target</code> را ایجاد کنید.
<br>
به‌عنوان راهنمایی می‌توانید از دستور <code>np.where</code> استفاده کنید. البته توجه کنید که راه‌های متنوعی برای انجام این کار وجود دارد.
</font>
</p>




In [64]:
train['Target'] = np.where(train['AdoptionSpeed'] == 4, False, True)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
حال می‌توانیم ستون‌هایی که به آن‌ها نیاز نداریم را از دیتافریم حذف کنیم. با توجه به این‌که ستون <code>AdoptionSpeed</code> را به ستون <code>Target</code> تبدیل کردیم، این ستون را حذف می‌کنیم، همچنین در ستون <code>Description</code> اطلاعات متنی داریم که روش پردازش و کار با این نوع داده‌ها را در فصل‌های بعدی بررسی خواهیم کرد، بنابراین این ستون را نیز حذف می‌کنیم.
</font>
</p>



In [65]:
train.drop(columns = ['AdoptionSpeed', 'Description'], inplace = True)
test.drop(columns = ['Description'], inplace = True)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در گام بعد به‌سراغ ویژگی‌های دسته‌ای می‌رویم. ابتدا مجدد نگاهی به جزئیات دیتافریم بی‌اندازیم.
</font>
</p>



In [66]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          10537 non-null  object
 1   Age           10537 non-null  int64 
 2   Breed1        10537 non-null  object
 3   Gender        10537 non-null  object
 4   Color1        10537 non-null  object
 5   Color2        10537 non-null  object
 6   MaturitySize  10537 non-null  object
 7   FurLength     10537 non-null  object
 8   Vaccinated    10537 non-null  object
 9   Sterilized    10537 non-null  object
 10  Health        10537 non-null  object
 11  Fee           10537 non-null  int64 
 12  PhotoAmt      10537 non-null  int64 
 13  Target        10537 non-null  bool  
dtypes: bool(1), int64(3), object(10)
memory usage: 1.1+ MB


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در خروجی قطعه‌کد بالا می توان دید که ستون‌های
   <code>Fee</code>، <code>PhotoAmt</code> و <code>Age</code> مقادیر عددی دارند و برای باقی ستون‌های مجموعه‌داده نیاز است، پیش‌پردازش لازم برای تبدیل آن‌ها به مقادیر عددی را انجام دهیم. پس به بررسی تعداد دسته‌های هر ویژگی و نوع آن‌ها می‌پردازیم
</font>
</p>


In [67]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

column: Type unique_values: 2
column: Breed1 unique_values: 165
column: Gender unique_values: 2
column: Color1 unique_values: 7
column: Color2 unique_values: 7
column: MaturitySize unique_values: 3
column: FurLength unique_values: 3
column: Vaccinated unique_values: 3
column: Sterilized unique_values: 3
column: Health unique_values: 3


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
 می‌دانیم ستون‌های <code>Health</code>، <code>FurLength</code> و <code>MaturitySize</code> از جنس متغیر‌های دسته‌ای ترتیبی هستند به این معنی که ترتیب خاصی بین مقادیر گروه‌ها وجود دارند. بنابراین می‌توانیم با استفاده از تابع <code>LabelEncoder</code> در کتابخانه‌ی <code>sklearn</code> به‌سادگی و بی‌دغدغه آن‌ها را به داده‌های عددی تبدیل کنیم.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
<span style="color:red"><b>توجه:</b></span> پیش‌پردازش‌هایی که بر روی نمونه‌های اعتبارسنجی/آزمون اعمال می‌کنید باید طبق آمارها یا اطلاعاتی باشد که از روی نمونه‌های آموزشی به دست آمده است. به‌عنوان مثال اگر می‌خواهید مقادیر گم‌شده‌ی یک ویژگی را با میانگین آن ویژگی پُر کنید، باید از میانگینی که طبق نمونه‌های آموزشی محاسبه‌شده استفاده کنید. به همین دلیل، در کدگذاری نیز برای نمونه‌های اعتبارسنجی/آزمون باید از تابع <code dir=ltr>transform()</code> به‌جای <code dir=ltr>fit_transform()</code> استفاده کنید.
</font>
</p>

In [68]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['MaturitySize'] = le.fit_transform(train['MaturitySize'])
test['MaturitySize'] = le.transform(test['MaturitySize'])

train['FurLength'] = le.fit_transform(train['FurLength'])
test['FurLength'] = le.transform(test['FurLength'])

train['Health'] = le.fit_transform(train['Health'])
test['Health'] = le.transform(test['Health'])

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
اما ستون‌های <code>Type</code>، <code>Breed1</code>، <code>Gender</code>، <code>Color1</code> ،<code>Color2</code> <code>Vaccinated</code> و <code>Sterilized</code>  از جنس متغیر‌های دسته‌ای اسمی هستند، به این معنی که ترتیب خاصی بین مقادیر گروه‌ها وجود ندارد. بنابراین برای این ویژگی‌ها نمی‌توانیم از <code>LabelEncoder</code> استفاده کنیم، زیرا این نوع داده‌ها از نظر ترتیبی تفاوتی با یک‌دیگر ندارند و اگر به‌سادگی به هر کدام از این ویژگی‌ها یک عدد اختصاص دهیم آن‌گاه به‌نوعی بین آن‌ها ترتیب قائل شدیم که نادرست است. برای حل این مسئله از روش کدگذاری دودویی استفاده می‌کنیم، به این معنی که به‌طور مثال به ازای ستون <code>color1</code> که ۷ مقدار یکتا <code dir=ltr>['Black', 'Brown', 'Cream', 'Gray', 'Golden', 'White', 'Yellow']</code> را دارد ۳ ستون اضافه می‌کنیم زیرا در هر ستون می‌توانیم مقدار ۱ و ۰ داشته باشیم و برای این‌که ۷ دسته را پوشش دهیم حداقل به ۳ ستون نیاز داریم (<code dir=ltr>7 < 3^2</code>). حال به هرکدام از این دسته‌ها عددی اختصاص می‌دهیم و ستون‌های مربوط به کد دودویی آن عدد را پر می‌کنیم.
<br>
    تابع <code>BinaryEncoder</code> از کتابخانه‌ی <code>category_encoders</code> این عملیات را انجام می‌دهد.
        <br>
    توجه داشته باشید در ستون <code>Gender</code> و <code>Type</code> به علت این‌که فقط ۲ دسته داریم می‌توانیم با تابع <code>LabelEncoder</code> تبدیل را انجام دهیم.
    </font>
</p>

In [69]:
train['Type'] = le.fit_transform(train['Type'])
test['Type'] = le.transform(test['Type'])

train['Gender'] = le.fit_transform(train['Gender'])
test['Gender'] = le.transform(test['Gender'])

In [70]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = binary_encoder.fit_transform(train['Breed1'])
test_binary_Breed1 = binary_encoder.transform(test['Breed1'])

train_binary_Color1 = binary_encoder.fit_transform(train['Color1'])
test_binary_Color1 = binary_encoder.transform(test['Color1'])

train_binary_Color2 = binary_encoder.fit_transform(train['Color2'])
test_binary_Color2 = binary_encoder.transform(test['Color2'])

train_binary_Vaccinated = binary_encoder.fit_transform(train['Vaccinated'])
test_binary_Vaccinated = binary_encoder.transform(test['Vaccinated'])

train_binary_Sterilized = binary_encoder.fit_transform(train['Sterilized'])
test_binary_Sterilized = binary_encoder.transform(test['Sterilized'])

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
حال ستون‌های ساخته شده برای هر متغیر را به مجموعه‌داده اصلی اضافه می‌کنیم و این ستون‌ها را حذف می‌کنیم.
</font>
</p>

In [71]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

test = pd.concat([test, test_binary_Breed1, test_binary_Color1, test_binary_Color2, test_binary_Vaccinated, test_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
test.drop(columns = columns, inplace = True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Type          10537 non-null  int64
 1   Age           10537 non-null  int64
 2   Gender        10537 non-null  int64
 3   MaturitySize  10537 non-null  int64
 4   FurLength     10537 non-null  int64
 5   Health        10537 non-null  int64
 6   Fee           10537 non-null  int64
 7   PhotoAmt      10537 non-null  int64
 8   Target        10537 non-null  bool 
 9   Breed1_0      10537 non-null  int64
 10  Breed1_1      10537 non-null  int64
 11  Breed1_2      10537 non-null  int64
 12  Breed1_3      10537 non-null  int64
 13  Breed1_4      10537 non-null  int64
 14  Breed1_5      10537 non-null  int64
 15  Breed1_6      10537 non-null  int64
 16  Breed1_7      10537 non-null  int64
 17  Color1_0      10537 non-null  int64
 18  Color1_1      10537 non-null  int64
 19  Color1_2      10537 non-n

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نرمال سازی داده‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
 یکی از اعمال مهم پیش‌پردازش، تغییر مقیاس ویژگی‌ها به‌صورت توزیع نرمال است که اصطلاحاً به آن نرمال‌سازی (Normalization) می‌گوییم.
 نرمال‌سازی به کاهش تغییرات شدید در وزن‌ها کمک کرده و باعث سرعت بالاتر همگرایی مدل می‌شود.
برای انجام نرمال‌سازی در این تمرین باید اعداد هر ویژگی را طوری تغییر دهیم که میانگین آن‌ها <code>0</code> شود و واریانس <code>1</code> داشته باشند، این‌کار را می‌توانیم با استفاده از فرمول زیر انجام دهیم.
برای سری داده‌ی <code>X</code> (که در اینجا مقادیر هر ستون می‌باشد) به طوری‌که <code>X = [x_1, x_2, ..., x_n]</code>، با کم کردن میانگین از هر نمونه‌ی داده (<code>x_i</code>) و تقسیم آن بر انحراف معیار داده‌ها (سیگما) می‌توانیم سری داده‌ی نرمال‌شده را به‌دست آوریم.
</font>
</p>

$$ Z = \frac{x_i - \bar{x}}{\sigma} $$

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
دقت داشته باشید، از آنجا که در واقع ما فقط اطلاعات مربوط به مجموعه‌داده‌ی آموزش را در هنگام ساخت مدل داریم، برای نرمال‌سازی نمونه‌های آزمون نیز باید از میانگین و انحراف معیار نمونه‌های آموزش استفاده کنیم. همچنین حتما از تابع میانگین و انحراف معیار کتابخانه <code>pandas</code> استفاده کنید.
</font>
</p>

In [72]:
target = train['Target']
train.drop(columns='Target', inplace=True)

In [73]:
for column in train.columns:
  mean = train[column].mean() #(Mean of train_data[column])
  std = train[column].std() #(Standard Deviation of train_data[column])
  train[column] = (train[column]- mean)/std
  test[column]=  (test[column] - mean)/std

test.head()

,Type,Age,Gender,MaturitySize,FurLength,Health,Fee,PhotoAmt,Breed1_0,Breed1_1,...,Color1_0,Color1_1,Color1_2,Color2_0,Color2_1,Color2_2,Vaccinated_0,Vaccinated_1,Sterilized_0,Sterilized_1
0,0.866680,-0.501042,-0.879677,-0.226574,0.790516,-0.194365,-0.29796,-0.830874,-0.073746,-0.184801,...,-0.521041,-0.861486,0.761850,-0.520158,0.777326,0.637933,-1.151487,0.386892,-0.730867,0.360165
1,-1.153719,0.015102,-0.879677,-0.226574,-0.845318,4.568489,-0.29796,-0.830874,-0.073746,-0.184801,...,-0.521041,1.160675,-1.312469,1.922312,0.777326,-1.567414,0.868360,-2.584453,1.368109,-2.776244
2,-1.153719,-0.449427,-0.879677,1.629890,0.790516,-0.194365,-0.29796,-0.512200,-0.073746,-0.184801,...,-0.521041,-0.861486,0.761850,-0.520158,-1.286339,0.637933,-1.151487,0.386892,-0.730867,0.360165
3,0.866680,-0.501042,-0.879677,-0.226574,-0.845318,-0.194365,-0.29796,0.443820,-0.073746,-0.184801,...,-0.521041,1.160675,0.761850,-0.520158,0.777326,0.637933,0.868360,0.386892,-0.730867,0.360165
4,0.866680,0.015102,-0.879677,-0.226574,-2.481151,-0.194365,-0.29796,-0.512200,-0.073746,-0.184801,...,1.919052,1.160675,-1.312469,-0.520158,0.777326,0.637933,0.868360,-2.584453,1.368109,-2.776244


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که در درسنامه‌های قبلی گفتیم برای ارزیابی و بهبود مدل بخشی از داده‌های آموزش را به‌عنوان داده‌های اعتبارسنجی جدا می‌کنیم. برای این‌کار می‌توانید از تابع <code>train_test_split</code> کتابخانه‌ی <code>sklearn</code> استفاده کنید. مستندات این تابع را می‌توانید در  <a href=https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html> این لینک </a> پیدا کنید. توجه داشته باشید در این مسئله <code>0.1</code> از داده‌های آموزش را به داده‌های اعتبارسنجی اختصاص می‌دهیم و <code>0.9</code> باقی مانده را برای آموزش مدل استفاده می‌کنیم.

</font>
</p>




In [74]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.1)

print('Train examples:', len(X_train), len(y_train))
print('Validation examples:', len(X_valid), len(y_valid))
print('Test examples:',  len(test))

Train examples: 9483 9483
Validation examples: 1054 1054
Test examples: 1000


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساخت شبکه‌ی عصبی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
پس از انجام پیش‌پرداز‌های مورد نیاز نوبت به ساخت و آموزش مدل رسیده است.
در این مسئله از شما می‌خواهیم ابتدا یک لایه ورودی با ابعاد مناسب مسئله که همان تعداد ویژگی‌های مجموعه‌داده است بسازید. سپس ۳ لایه‌ی متراکم با تعداد نورون‌های ۵۰۰۰، ۱۰۰۰ و ۵۰۰ (به ترتیب از راست به چپ) با تابع‌ فعال‌ساز  <code>relu</code> ایجاد کنید و در نهایت یک لایه‌ی متراکم خروجی با تعداد نورون مناسب و تابع فعال‌ساز  <code>sigmoid</code> اضافه کنید.
</font>
</p>

In [75]:
model = keras.Sequential() # define a Sequential model
model.add(keras.layers.Input(shape=(26,))) # : add Input Layer with proper Shape
model.add(keras.layers.Dense(5000, activation='relu')) # : add Dense Layer with 5000 neurons and relu activation function
model.add(keras.layers.Dense(1000, activation='relu')) # : add Dense Layer with 1000 neurons and relu activation function
model.add(keras.layers.Dense(500, activation='relu')) # : add Dense Layer with 500 neurons and relu activation function
model.add(keras.layers.Dense(1, activation='sigmoid')) # : add Dense Layer as Output Layer with sigmoid activation function

In [76]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 5000)           │       135,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,637,001 (21.50 MB)

 Trainable params: 5,637,001 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش شبکه‌ی عصبی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در ادامه شبکه را با تابع هزینه و بهینه‌ساز تعریف‌شده <code>compile</code> می‌کنیم. (درباره‌ی توابع هزینه‌ و بهینه‌سازهای مختلف در فصل بعد کامل صحبت خواهیم کرد)</font>
</p>

In [77]:
model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['f1_score'])

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این مرحله باید مدل ساخته شده را با اندازه‌ی دسته ۱۲۸ (<code>BATCH_SIZE</code>) و به تعداد دور ۱۰ (<code>epochs</code>) آموزش دهید.
</font>
</p>




In [78]:
print(X_train.shape)
print(y_train.shape)
print(test.shape)

(9483, 26)
(9483,)
(1000, 26)


In [79]:
epochs = 15
BATCH_SIZE = 128

history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - f1_score: 0.8480 - loss: 0.5723 - val_f1_score: 0.8430 - val_loss: 0.5328
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - f1_score: 0.8438 - loss: 0.5209 - val_f1_score: 0.8430 - val_loss: 0.5211
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - f1_score: 0.8427 - loss: 0.5058 - val_f1_score: 0.8430 - val_loss: 0.5358
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - f1_score: 0.8488 - loss: 0.4905 - val_f1_score: 0.8430 - val_loss: 0.5164
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - f1_score: 0.8431 - loss: 0.4873 - val_f1_score: 0.8430 - val_loss: 0.5142
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - f1_score: 0.8437 - loss: 0.4744 - val_f1_score: 0.8430 - val_loss: 0.5274
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - f1_score: 0.8468 - loss: 0.4709 - val_f1_score: 0.8430 - val_loss: 0.5182
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - f1_score: 0.8462 - loss: 0.4518 - val_f1_score: 0.8430 - v

75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - f1_score: 0.8461 - loss: 0.4013 - val_f1_score: 0.8430 - val_loss: 0.6004
Epoch 15/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - f1_score: 0.8452 - loss: 0.3784 - val_f1_score: 0.8430 - val_loss: 0.6069



<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی برای داده‌های آزمون و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
شبکه‌ی خود را روی مجموعه‌داده‌ی <code>test</code>  آزمایش کنید و خروجی را در یک دیتافریم (<code>dataframe</code>) ذخیره کنید. دقت کنید مقادیر مجاز مقادیر دودویی (<code>False</code>/<code>True</code>) می‌باشد.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید معیار ارزیابی مدل شما در این تمرین معیار <code>F1 score</code> است، که می‌توانید مستندات آن را در <a href = 'https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html'>این لینک</a> مطالعه نمایید. لازم به ذکر است که از حالت میانگین‌گیری <code>"weighted"</code> این معیار استفاده خواهد شد.
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>70</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
  
|ستون|توضیحات|
|:------:|:---:|
|<code>Target</code>|آیا حیوان‌خانگی خانه جدید پیدا کرده است یا خیر|
  
</font>
</div>
</center>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اسم دیتافریم باید <code>submission</code> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش‌ شما را ارزیابی کند.
    <br>
    این دیتافریم تنها شامل ۱ ستون با اسم <code>Target</code> است و ۱۰۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در دیتافریم <code>test</code> شما باید یک مقدار پیش‌بینی‌شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، اعداد ستون <code>Target</code> ممکن است متفاوت باشد و این فقط یک مثال است.
</font>
</p>

<center>
<div dir=ltr style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>Target</code>|
|:----:|:-----:|
|0|<code>True</code>|
|1|<code>True</code>|
|2|<code>False</code>|
|3|<code>False</code>|
|4|<code>True</code>|

</font>
</div>
</center>

In [80]:
submission = model.predict(test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [81]:
submission

array([[9.88773942e-01],
       [1.78775415e-02],
       [6.41041100e-01],
       [8.86288404e-01],
       [8.65341663e-01],
       [3.45071942e-01],
       [8.94373059e-01],
       [5.58623612e-01],
       [5.14874399e-01],
       [8.51013184e-01],
       [7.65954614e-01],
       [9.14887011e-01],
       [6.23655200e-01],
       [1.47572691e-02],
       [5.86360693e-02],
       [9.01915014e-01],
       [5.29168010e-01],
       [8.44283819e-01],
       [1.63925603e-01],
       [7.57047892e-01],
       [3.30887616e-01],
       [7.69337058e-01],
       [8.26704562e-01],
       [4.04429227e-01],
       [5.03525674e-01],
       [9.99708652e-01],
       [3.79357368e-01],
       [9.19379175e-01],
       [5.30199945e-01],
       [5.90533972e-01],
       [2.29955345e-01],
       [9.49474156e-01],
       [5.82218505e-02],
       [9.05206084e-01],
       [6.07234776e-01],
       [8.64858568e-01],
       [6.64085209e-01],
       [8.06596637e-01],
       [8.99595022e-01],
       [9.42414284e-01],


In [82]:
submission = submission.flatten()
submission = pd.DataFrame(submission, columns=['Target'])
submission['Target'] = submission['Target'] > 0.5

print(submission.head(10))

   Target
0    True
1   False
2    True
3    True
4    True
5   False
6    True
7    True
8    True
9    True


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد. همچنین اگر از گوگل کولب استفاده می‌کنید، در صورت نیاز به پشتیبانی حتماً آخرین نسخه از نت‌بوک را به‌صورت دستی دانلود کرده و داخل فایل ارسالی قرار دهید یا لینک کولب را با ما به‌اشتراک بگذارید.
</font>
</p>

In [84]:
import zipfile
import joblib
import numpy as np
import json
import os

if not os.path.exists(os.path.join(os.getcwd(), 'new_home.ipynb')):
    %notebook -e new_home.ipynb

test.to_csv("test.csv")

model_info=[]
for layer in model.layers:
  if layer.__class__.__name__=="Dense":
    model_info.append({
        "name": layer.__class__.__name__,
        "units":layer.units,
        "activation":layer.get_config()["activation"]
    })
  else:
    model_info.append({"name": layer.__class__.__name__})

with open("model_info.json", "w") as f:
    json.dump(model_info, f)


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)

file_names = ["test.csv",'submission.csv', 'model_info.json', 'new_home.ipynb']
compress(file_names)

File Paths:
['test.csv', 'submission.csv', 'model_info.json', 'new_home.ipynb']
